<a href="https://colab.research.google.com/github/leemthompo/notebook-tests/blob/main/esql_811_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install client, import libraries

In [38]:
!pip install elasticsearch
from elasticsearch import Elasticsearch
import getpass

## Create client instance


In [40]:
CLOUD_ID = getpass.getpass(prompt="Enter CLOUD ID:")
API_KEY= getpass.getpass(prompt="Enter API key")


client = Elasticsearch(
  CLOUD_ID,
  api_key=API_KEY
)

client.info()

Enter CLOUD ID:··········
Enter password··········


ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '3eec50b2ab254ac1b027c6f0be6bad16', 'cluster_uuid': '4HPZ8CJvRm6FQ2RwAFo_Cw', 'version': {'number': '8.11.0-SNAPSHOT', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '454cd35d33aafc161b4f7238d63777e71814d834', 'build_date': '2023-10-16T15:48:25.683294483Z', 'build_snapshot': True, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Create sample index

In [42]:
# Create the 'library' index (assuming default settings and mappings)
if not client.indices.exists(index='library'):
    client.indices.create(index='library')

# Data to be indexed
books = [
    {
        "author": "Peter F. Hamilton",
        "name": "Pandora's Star",
        "page_count": 768,
        "release_date": "2004-03-02T00:00:00.000Z"
    },
    {
        "author": "Vernor Vinge",
        "name": "A Fire Upon the Deep",
        "page_count": 613,
        "release_date": "1992-06-01T00:00:00.000Z"
    },
    {
        "author": "Frank Herbert",
        "name": "Dune",
        "page_count": 604,
        "release_date": "1965-06-01T00:00:00.000Z"
    },
    {
        "author": "Alastair Reynolds",
        "name": "Revelation Space",
        "page_count": 585,
        "release_date": "2000-03-15T00:00:00.000Z"
    },
    {
        "author": "James S.A. Corey",
        "name": "Leviathan Wakes",
        "page_count": 561,
        "release_date": "2011-06-02T00:00:00.000Z"
    }
]

# Index the data
for book in books:
    client.index(index='library', body=book)

print("Data indexed successfully!")

Data indexed successfully!


## Test Query 1

In [48]:
# Your query
data = {
    "query": "FROM library | KEEP author, name, page_count, release_date | SORT page_count DESC | LIMIT 5"
}

headers = {"Content-Type": "application/json", "Accept": "application/json"}

# Execute the query
response = client.perform_request('POST', '/_query?format=txt', body=data, headers=headers)

print(response.body)



     author      |        name        |  page_count   |      release_date      
-----------------+--------------------+---------------+------------------------
Peter F. Hamilton|Pandora's Star      |768            |2004-03-02T00:00:00.000Z
Peter F. Hamilton|Pandora's Star      |768            |2004-03-02T00:00:00.000Z
Peter F. Hamilton|Pandora's Star      |768            |2004-03-02T00:00:00.000Z
Vernor Vinge     |A Fire Upon the Deep|613            |1992-06-01T00:00:00.000Z
Vernor Vinge     |A Fire Upon the Deep|613            |1992-06-01T00:00:00.000Z



## Test query 2

In [45]:
# Your query
data = {
    "query": """
        FROM library
        | EVAL year = DATE_TRUNC(1 YEARS, release_date)
        | STATS MAX(page_count) BY year
        | SORT year
        | LIMIT 5
    """
}


headers = {"Content-Type": "application/json", "Accept": "application/json"}


# Execute the query
response = client.perform_request('POST', '/_query?format=txt', body=data, headers=headers)

print(response.body)

['MAX(page_count)|          year          ',
 '---------------+------------------------',
 '604            |1965-01-01T00:00:00.000Z',
 '613            |1992-01-01T00:00:00.000Z',
 '585            |2000-01-01T00:00:00.000Z',
 '768            |2004-01-01T00:00:00.000Z',
 '561            |2011-01-01T00:00:00.000Z']